### Unsloth 加速
#### 使 Llama-3、Mistral、Phi-4 和 Gemma 等大型语言模型的微调速度提高了 2 倍，使用的内存减少了 70%，并且准确性没有下降！

DeepSeek 开发了 GRPO （Group Relative Policy Optimization） 来训练他们的 R1 推理模型。这种 RL 技术无需价值函数模型即可有效地优化响应，与 PPO （近端策略优化） 相比，减少了内存和计算成本。

建议将 GRPO 应用于参数至少为 1.5B 的模型，以正确生成思维标记，因为较小的模型可能无法正确生成。

###  Installation dependence

In [1]:
!pip install unsloth vllm
!pip install --upgrade pillow
!pip install diffusers
# Temporarily install a specific TRL nightly version
!pip install git+https://github.com/huggingface/trl.git@e95f9fb74a3c3647b86f251b7e230ec51c64b72b

  Cloning https://github.com/huggingface/trl.git (to revision e95f9fb74a3c3647b86f251b7e230ec51c64b72b) to /tmp/pip-req-build-0f3gl1by
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/trl.git /tmp/pip-req-build-0f3gl1by
  Running command git rev-parse -q --verify 'sha^e95f9fb74a3c3647b86f251b7e230ec51c64b72b'
  Running command git fetch -q https://github.com/huggingface/trl.git e95f9fb74a3c3647b86f251b7e230ec51c64b72b
  Running command git checkout -q e95f9fb74a3c3647b86f251b7e230ec51c64b72b
  Resolved https://github.com/huggingface/trl.git to commit e95f9fb74a3c3647b86f251b7e230ec51c64b72b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for trl: filename=trl-0.15.0.dev0-py3-none-any.whl size=314513 sha256=c753e3b67e0a6e188690a00bcfc03c784634e3f6e66a19a01699e4b1614ba51c
  Stored in directory: /home/olivia/.cache/pip/wheels/1f/46/e2/1d706f988ec4990b

### install Unsloth

使用PatchFastRL对HuggingFace的trl进行修改，注入自己的加速代码。目前只支持单机版的加速。分布式版尚未支持。

In [1]:
from unsloth import FastLanguageModel, PatchFastRL
# Use `PatchFastRL` before all functions to patch GRPO and other RL algorithms!
PatchFastRL("GRPO", FastLanguageModel) 

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/anaconda/envs/sglang/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 02-26 07:17:24 __init__.py:190] Automatically detected platform cuda.


2025-02-26 07:17:25,101	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Load the phi-4 model and its tokenizer

In [ ]:
from unsloth import is_bfloat16_supported
import torch
max_seq_length = 512 # Can increase for longer reasoning traces
lora_rank = 16 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "microsoft/phi-4", # Load up `Phi-4 14B`, and set parameters
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.4, # Reduce if out of memory  #0.7
    device_map='cuda:0',
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["gate_proj", "up_proj", "down_proj",], #要微调的模块（默认包括 ）
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

Unsloth: Switching from Unsloth dynamic quant to normal quant since
we do not yet support fast inference for unsloth/phi-4-unsloth-bnb-4bit
==((====))==  Unsloth 2025.2.12: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA A100 80GB PCIe. Max memory: 79.151 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/phi-4-bnb-4bit with actual GPU utilization = 35.96%
Unsloth: Your GPU has CUDA compute capability 8.0 with VRAM = 79.15 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 512. Num Sequences = 288.
Unsloth: vLLM's KV Cache can use up to 18.49 GB. Also swap space = 6 GB.
INFO 02-26 07:18:11 config.py:542] This model supports multiple tasks: {'gene

[W226 07:18:14.952978546 CUDAAllocatorConfig.h:28] Warning: expandable_segments not supported on this platform (function operator())


INFO 02-26 07:18:14 model_runner.py:1110] Starting to load model unsloth/phi-4-bnb-4bit...
INFO 02-26 07:18:16 loader.py:1102] Loading weights with BitsAndBytes quantization.  May take a while ...
INFO 02-26 07:18:16 weight_utils.py:252] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [02:54<02:54, 174.47s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [05:53<00:00, 177.23s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [05:53<00:00, 176.81s/it]

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:02<00:02,  2.64s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:06<00:00,  3.08s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:06<00:00,  3.02s/it]



INFO 02-26 07:24:17 model_runner.py:1115] Loading model weights took 8.4920 GB
INFO 02-26 07:24:17 punica_selector.py:18] Using PunicaWrapperGPU.
INFO 02-26 07:24:29 worker.py:267] Memory profiling takes 12.20 seconds
INFO 02-26 07:24:29 worker.py:267] the current vLLM instance can use total_gpu_memory (79.15GiB) x gpu_memory_utilization (0.36) = 28.46GiB
INFO 02-26 07:24:29 worker.py:267] model weights take 8.49GiB; non_torch_memory takes 13.01GiB; PyTorch activation peak memory takes 1.04GiB; the rest of the memory reserved for KV Cache is 5.92GiB.
INFO 02-26 07:24:29 executor_base.py:110] # CUDA blocks: 1939, # CPU blocks: 1966
INFO 02-26 07:24:29 executor_base.py:115] Maximum concurrency for 512 tokens per request: 60.59x
INFO 02-26 07:24:35 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error o

Capturing CUDA graph shapes: 100%|██████████| 39/39 [00:41<00:00,  1.06s/it]

INFO 02-26 07:25:17 model_runner.py:1562] Graph capturing finished in 41 secs, took 6.36 GiB
INFO 02-26 07:25:17 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 60.18 seconds



Not an error, but Unsloth cannot patch Attention layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Not an error, but Unsloth cannot patch O projection layer with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2025.2.12 patched 40 layers with 0 QKV layers, 0 O layers and 40 MLP layers.


### 奖励函数定义和数据准备 
<a name="Data"></a

可能需要等待 150 到 200 个步骤奖励才会真正增加。为了获得不错的结果，您可能需要交易至少 12 小时（这就是 GRPO 的运作方式），这不是强制性的。


为了获得最佳结果，至少要有 500 行数据。可以尝试使用 10 行数据，但最好有更多数据。


每次训练运行总是不同的，具体取决于模型、数据、奖励函数/验证器等。


为了达到演示效果，参考 [@willccbb](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb) 

In [ ]:
import re
from datasets import load_dataset, Dataset

# Load and prep dataset
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    # 将输入的文本字符串按照"<answer>"进行分割，得到一个列表
    # 取列表中的最后一个元素，即包含答案的部分
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    # 使用strip()方法去除答案字符串两端的空白字符（包括空格、换行符等）
    # 返回处理后的答案字符串
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

# uncomment middle messages for 1-shot prompting
def get_gsm8k_questions(split = "train") -> Dataset:
    # 加载GSM8K数据集，指定split为"train"或"test"
    data = load_dataset('openai/gsm8k', 'main')[split] # type: ignore
    # 使用map函数对数据集进行转换，将每个样本的'question'和'answer'字段转换为特定格式
    data = data.map(lambda x: { # type: ignore
        'prompt': [ # 构造对话提示，包含系统提示和用户问题
            {'role': 'system', 'content': SYSTEM_PROMPT}, # 系统提示，通常包含对话的背景信息或规则
            {'role': 'user', 'content': x['question']} # 用户问题，从原始数据中提取
        ],
        'answer': extract_hash_answer(x['answer']) # 提取答案的哈希值，具体实现见extract_hash_answer函数
    }) # type: ignore
    return data # type: ignore

dataset = get_gsm8k_questions()


############  奖励函数 ############

def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    # 从completions列表中提取每个完成内容的第一个元素的content部分
    responses = [completion[0]['content'] for completion in completions]
    # 从prompts列表中提取最后一个元素的content部分，作为问题
    q = prompts[0][-1]['content']
    # 对每个响应内容调用extract_xml_answer函数，提取出答案部分
    extracted_responses = [extract_xml_answer(r) for r in responses]
    # 打印分隔线、问题、标准答案、第一个响应内容和第一个提取的答案
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    # 比较提取的答案与标准答案，如果相同则返回2.0，否则返回0.0，形成列表返回
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    # 从输入的completions列表中提取每个完成项的第一个内容的文本部分
    responses = [completion[0]['content'] for completion in completions]
    # 对每个响应文本调用extract_xml_answer函数，提取出XML格式的答案
    extracted_responses = [extract_xml_answer(r) for r in responses]
    # 对每个提取出的答案进行判断，如果是数字则返回0.5，否则返回0.0
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    # 定义一个正则表达式模式，用于匹配特定格式的字符串
    # 该模式要求字符串以"<reasoning>"开始，然后是任意字符，接着是"</reasoning>"
    # 然后是"<answer>"，再接着是任意字符，最后是"</answer>"
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    # 从输入的completions列表中提取每个完成的内容
    # completions是一个列表，每个元素是一个字典，字典中包含一个键为"content"的项，其值为完成的内容
    responses = [completion[0]["content"] for completion in completions]
    # 使用正则表达式模式匹配每个响应内容
    # re.match函数尝试从字符串的开始位置匹配正则表达式
    # 如果匹配成功，返回一个匹配对象；否则返回None
    matches = [re.match(pattern, r) for r in responses]
    # 根据匹配结果生成奖励值列表
    # 如果匹配成功（match不为None），则奖励值为0.5；否则为0.0
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

### 训练模型 Train the model

和用Hugging face的trl是一样的。

In [4]:
from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 5e-5,#-6
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "paged_adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 6, # Decrease if out of memory
    max_prompt_length = 256,
    max_completion_length = 200,
    max_steps = 10,# 100
    save_steps = 250,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)



前 100 个步骤可能会获得 0 奖励

In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [ #以上定义的多个奖励函数
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset
)
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 7,473 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 1
\        /    Total batch size = 1 | Total steps = 10
 "-____-"     Number of trainable parameters = 44,236,800


-------------------- Question:
A concert ticket costs $40. Mr. Benson bought 12 tickets and received a 5% discount for every ticket bought that exceeds 10. How much did Mr. Benson pay in all? 
Answer:
476 
Response:
<reasoning>
To determine how much Mr. Benson paid, we need to calculate the cost of the first 10 tickets and then apply the discount for the remaining 2 tickets.

1. **Calculate the cost of the first 10 tickets:**
   - Each ticket costs $40.
   - Therefore, the cost for 10 tickets is \(10 \times 40 = 400\) dollars.

2. **Calculate the cost of the additional 2 tickets with a discount:**
   - Normally, each ticket costs $40.
   - He receives a 5% discount for each ticket beyond the 10th.
   - The discount per ticket is \(5\% \times 40 = 0.05 \times 40 = 2\) dollars.
   - Therefore, the discounted price per ticket is \(40 - 2 = 38\) dollars.
   - The cost for 2 discounted tickets is \(2 \times 38 = 76\) dollars.

 
Extracted:
<reasoning>
To determine how much Mr. Benson paid, 

Step,Training Loss,reward,reward_std,completion_length,kl,rewards / xmlcount_reward_func,rewards / soft_format_reward_func,rewards / strict_format_reward_func,rewards / int_reward_func,rewards / correctness_reward_func
1,0.000000,0.125000,0.000000,200.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.083333,0.064550,200.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.125000,0.000000,200.000000,0.000083,0.125000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.125000,0.000000,200.000000,0.000095,0.125000,0.000000,0.000000,0.000000,0.000000
5,0.000000,-0.128500,0.147544,195.833344,0.000089,-0.128500,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.125000,0.000000,200.000000,0.000080,0.125000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.125000,0.000000,200.000000,0.000141,0.125000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.503000,0.925907,195.500000,0.000229,0.086333,0.000000,0.000000,0.083333,0.333333
9,0.000000,0.125000,0.000000,200.000000,0.000110,0.125000,0.000000,0.000000,0.000000,0.000000
10,0.000000,0.125000,0.000000,200.000000,0.000173,0.125000,0.000000,0.000000,0.000000,0.000000


-------------------- Question:
Jane is trying to decide whether to buy a house or a trailer. A house costs $480,000 and a trailer costs $120,000. Each loan will be paid in monthly installments over 20 years. How much more is the monthly payment on the house compared to the trailer? 
Answer:
1500 
Response:
<reasoning>
To determine how much more the monthly payment on the house is compared to the trailer, we need to calculate the monthly loan payments for both the house and the trailer using the formula for monthly payments on an amortizing loan:

\[
M = P \frac{r(1 + r)^n}{(1 + r)^n - 1}
\]

Where:
- \( M \) is the monthly payment.
- \( P \) is the principal loan amount (the initial amount of the loan).
- \( r \) is the monthly interest rate (annual interest rate divided by 12).
- \( n \) is the number of payments (loan term in years multiplied by 12).

Assumptions:
- For this problem, we have not been provided with an interest rate. In real-world scenarios, the interest rate significa

In [ ]:
# 从trainer_stats对象中获取训练过程中的度量指标
metrics = trainer_stats.metrics
metrics

In [7]:
trainer.state.log_history

[{'loss': 0.0,
  'grad_norm': 0.0,
  'learning_rate': 5e-05,
  'rewards/xmlcount_reward_func': 0.125,
  'rewards/soft_format_reward_func': 0.0,
  'rewards/strict_format_reward_func': 0.0,
  'rewards/int_reward_func': 0.0,
  'rewards/correctness_reward_func': 0.0,
  'reward': 0.125,
  'reward_std': 0.0,
  'completion_length': 200.0,
  'kl': 0.0,
  'epoch': 0.00013381506757660912,
  'step': 1},
 {'loss': 0.0,
  'grad_norm': 0.08599156886339188,
  'learning_rate': 4.849231551964771e-05,
  'rewards/xmlcount_reward_func': 0.0833333358168602,
  'rewards/soft_format_reward_func': 0.0,
  'rewards/strict_format_reward_func': 0.0,
  'rewards/int_reward_func': 0.0,
  'rewards/correctness_reward_func': 0.0,
  'reward': 0.0833333358168602,
  'reward_std': 0.06454972922801971,
  'completion_length': 200.0,
  'kl': 0.0,
  'epoch': 0.00026763013515321824,
  'step': 2},
 {'loss': 0.0,
  'grad_norm': 6.406837928807363e-05,
  'learning_rate': 4.415111107797445e-05,
  'rewards/xmlcount_reward_func': 0.125

### 部署推理 Inference

使用没GRPO训练过的模型进行推理的代码如下:

In [9]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "100以内的质数有哪些?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:07<00:00,  7.69s/it, est. speed input: 2.47 toks/s, output: 17.57 toks/s]


'100以内的质数是只有1和自身两个正整数因子的自然数。它们是：\n\n1, 2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97\n\n注意：1不是质数，它通常被排除在质数的定义之外。'

In [10]:
print(output)

100以内的质数是只有1和自身两个正整数因子的自然数。它们是：

1, 2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97

注意：1不是质数，它通常被排除在质数的定义之外。


为了使用GRPO训练后的模型，我们需要先保持训练过的LoRA权重。

In [11]:
model.save_lora("grpo_saved_lora")

使用GRPO训练过的模型进行部署推理:

In [14]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "100以内的质数有哪些?用中文回答"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:30<00:00, 30.73s/it, est. speed input: 1.66 toks/s, output: 15.03 toks/s]


'<reasoning>\n要找到100以内的质数，我们首先需要了解什么是质数。质数是只能被1和它本身整除的自然数，且不等于1。因此，质数必须大于1。\n\n我们从2开始，因为2是最小的质数。然后，我们逐个检查每个整数是否符合质数的定义：\n\n- 2是质数，因为它只能被1和2整除。\n- 3是质数，因为它只能被1和3整除。\n- 4不是质数，因为它可以被2整除。\n- 5是质数，因为它只能被1和5整除。\n- 6不是质数，因为它可以被2和3整除。\n- 7是质数，因为它只能被1和7整除。\n- 8不是质数，因为它可以被2整除。\n- 9不是质数，因为它可以被3整除。\n- 10不是质数，因为它可以被2和5整除。\n- 11是质数，因为它只能被1和11整除。\n\n我们继续这个过程，一直到100。在此过程中，我们需要排除所有非质数（合数），即那些能被小于自身的自然数整除的数。\n\n在这个范围内，可以通过除以小于其平方根的质数来测试一个数是否为质数。如果一个数不能被任何这些质数整除，那么它就是质数。\n\n通过这个方法，我们可以列出100以内的质数：\n2, 3, 5,'

In [15]:
print(output)

<reasoning>
要找到100以内的质数，我们首先需要了解什么是质数。质数是只能被1和它本身整除的自然数，且不等于1。因此，质数必须大于1。

我们从2开始，因为2是最小的质数。然后，我们逐个检查每个整数是否符合质数的定义：

- 2是质数，因为它只能被1和2整除。
- 3是质数，因为它只能被1和3整除。
- 4不是质数，因为它可以被2整除。
- 5是质数，因为它只能被1和5整除。
- 6不是质数，因为它可以被2和3整除。
- 7是质数，因为它只能被1和7整除。
- 8不是质数，因为它可以被2整除。
- 9不是质数，因为它可以被3整除。
- 10不是质数，因为它可以被2和5整除。
- 11是质数，因为它只能被1和11整除。

我们继续这个过程，一直到100。在此过程中，我们需要排除所有非质数（合数），即那些能被小于自身的自然数整除的数。

在这个范围内，可以通过除以小于其平方根的质数来测试一个数是否为质数。如果一个数不能被任何这些质数整除，那么它就是质数。

通过这个方法，我们可以列出100以内的质数：
2, 3, 5,
